In [2]:
import numpy as np
import cv2
import sys
import time


In [26]:
ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

def draw_axis(img, rvec, tvec, camera_matrix, dist_coeffs, length):
    axis_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,length]]).reshape(-1,3)

    # Ensure correct formatting of rotation vector
    rvec = rvec.reshape((3,))

    # Project axis points to the image plane
    img_points, _ = cv2.projectPoints(axis_points, rvec, tvec, camera_matrix, dist_coeffs)

    # Convert image points to integers
    img_points = np.round(img_points).astype(int)

    # Draw lines
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[1].ravel()), (0,0,255), 2)  # x-axis (red)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[2].ravel()), (0,255,0), 2)  # y-axis (green)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[3].ravel()), (255,0,0), 2)  # z-axis (blue)

    return img

def aruco_display(corners, ids, rejected, image):
    
	if len(corners) > 0:
		
		ids = ids.flatten()
		
		for (markerCorner, markerID) in zip(corners, ids):
			
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners
			
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
			
			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			print("[Inference] ArUco marker ID: {}".format(markerID))
			
	return image



def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Call aruco_display to draw markers and their IDs
    frame = aruco_display(corners, ids, rejected_img_points, frame)

    if len(corners) > 0:
        for i in range(0, len(ids)):
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients, distortion_coefficients)

            draw_axis(frame, rvec, tvec, matrix_coefficients, distortion_coefficients, 0.1)  # Pass rvec and tvec to draw_axis

    return frame

aruco_type = "DICT_ARUCO_ORIGINAL"

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))


cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)



while cap.isOpened():
    
    ret, img = cap.read()
    
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)

    cv2.imshow('Estimated Pose', output)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] 

[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt2'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt2'. Sequence item with index 0 has a wrong type


In [1]:
import numpy as np
import cv2

ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL}

def draw_axis(img, rvec, tvec, camera_matrix, dist_coeffs, length):
    axis_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,length]]).reshape(-1,3)

    # Ensure correct formatting of rotation vector
    rvec = rvec.reshape((3,))

    # Project axis points to the image plane
    img_points, _ = cv2.projectPoints(axis_points, rvec, tvec, camera_matrix, dist_coeffs)

    # Convert image points to integers
    img_points = np.round(img_points).astype(int)

    # Draw lines
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[1].ravel()), (0,0,255), 2)  # x-axis (red)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[2].ravel()), (0,255,0), 2)  # y-axis (green)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[3].ravel()), (255,0,0), 2)  # z-axis (blue)

    return img

def aruco_display(corners, ids, rejected, image):
    
	if len(corners) > 0:
		
		ids = ids.flatten()
		
		for (markerCorner, markerID) in zip(corners, ids):
			
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners
			
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
			
			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			print("[Inference] ArUco marker ID: {}".format(markerID))
			
	return image



def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Call aruco_display to draw markers and their IDs
    frame = aruco_display(corners, ids, rejected_img_points, frame)

    if len(corners) > 0:
        for i in range(0, len(ids)):
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.175, matrix_coefficients, distortion_coefficients)

            draw_axis(frame, rvec, tvec, matrix_coefficients, distortion_coefficients, 0.1)  # Pass rvec and tvec to draw_axis

    return frame

def distance_between_markers(tvec1, tvec2):
    # Calculate the Euclidean distance between two translation vectors
    distance = np.linalg.norm(tvec1 - tvec2)
    return distance

aruco_type = "DICT_ARUCO_ORIGINAL"

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))

# Define the IDs of the two markers you are interested in
marker1_id = 0;
marker2_id = 153;

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Iinitializing Coordinate Vectors
tvec1 = ([[0], [0], [0]])
tvec2 = ([[0], [0], [0]])

# While loop for computer vision, currently calculates distance between markers only
while cap.isOpened():
    
    ret, img = cap.read()
    
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)
    
    cv2.imshow('Estimated Pose', output)
    
    # Relative Positions
    tvec_marker1 =  np.array(tvec1)   # Translation vector of marker 1
    tvec_marker2 = np.array(tvec2)    # Translation vector of marker 2

    distance = distance_between_markers(tvec_marker1, tvec_marker2)
    print("Distance between markers:", distance)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance b

Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
Dis

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 1

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
Distance between markers: 0.0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 

In [52]:
import numpy as np
import cv2

ARUCO_DICT = {
    "DICT_4X4_50": cv2.aruco.DICT_4X4_50,
    "DICT_4X4_100": cv2.aruco.DICT_4X4_100,
    "DICT_4X4_250": cv2.aruco.DICT_4X4_250,
    "DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
    "DICT_5X5_50": cv2.aruco.DICT_5X5_50,
    "DICT_5X5_100": cv2.aruco.DICT_5X5_100,
    "DICT_5X5_250": cv2.aruco.DICT_5X5_250,
    "DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
    "DICT_6X6_50": cv2.aruco.DICT_6X6_50,
    "DICT_6X6_100": cv2.aruco.DICT_6X6_100,
    "DICT_6X6_250": cv2.aruco.DICT_6X6_250,
    "DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
    "DICT_7X7_50": cv2.aruco.DICT_7X7_50,
    "DICT_7X7_100": cv2.aruco.DICT_7X7_100,
    "DICT_7X7_250": cv2.aruco.DICT_7X7_250,
    "DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
    "DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
    "DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
    "DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
    "DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
    "DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

def draw_axis(img, rvec, tvec, camera_matrix, dist_coeffs, length):
    axis_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,length]]).reshape(-1,3)

    # Ensure correct formatting of rotation vector
    rvec = rvec.reshape((3,))

    # Project axis points to the image plane
    img_points, _ = cv2.projectPoints(axis_points, rvec, tvec, camera_matrix, dist_coeffs)

    # Convert image points to integers
    img_points = np.round(img_points).astype(int)

    # Draw lines
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[1].ravel()), (0,0,255), 2)  # x-axis (red)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[2].ravel()), (0,255,0), 2)  # y-axis (green)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[3].ravel()), (255,0,0), 2)  # z-axis (blue)

    return img

def aruco_display(corners, ids, rejected, image):
    if len(corners) > 0:
        ids = ids.flatten()
        for (markerCorner, markerID) in zip(corners, ids):
            corners = markerCorner.reshape((4, 2))
            (topLeft, topRight, bottomRight, bottomLeft) = corners
            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))
            cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
            cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
            cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
            cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
            cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (0, 255, 0), 2)
            print("[Inference] ArUco marker ID: {}".format(markerID))
    return image

def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict, parameters=parameters)

    if len(corners) > 0:
        for i in range(0, len(ids)):
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                            distortion_coefficients)

            cv2.aruco.drawDetectedMarkers(frame, corners)

            draw_axis(frame, rvec, tvec, matrix_coefficients, distortion_coefficients, 0.1)

    return frame

def distance_between_markers(tvec1, tvec2):
    distance = np.linalg.norm(tvec1 - tvec2)
    return distance

aruco_type = "DICT_ARUCO_ORIGINAL"

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters()

intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948, 0.18514, 0, 0))

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    ret, img = cap.read()
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)

    # Assuming you have two ArUco markers with known translation vectors tvec_marker1 and tvec_marker2
    tvec_marker1 = np.array(tvec1)  # Translation vector of marker 1
    tvec_marker2 = np.array(tvec2)  # Translation vector of marker 2

    distance = distance_between_markers(tvec_marker1, tvec_marker2)
    print("Distance between markers:", distance)

    cv2.imshow('Estimated Pose', output)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance b

Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance b

Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance b

Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance b

Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance between markers: 0.0
Distance b

In [7]:
import numpy as np
import cv2
from PIL import Image, ImageDraw

ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,}

def draw_axis(img, rvec, tvec, camera_matrix, dist_coeffs, length):
    axis_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,length]]).reshape(-1,3)

    # Project axis points to the image plane
    img_points, _ = cv2.projectPoints(axis_points, rvec, tvec, camera_matrix, dist_coeffs)

    # Convert image points to integers
    img_points = np.round(img_points).astype(int)

    # Draw lines
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[1].ravel()), (0,0,255), 2)  # x-axis (red)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[2].ravel()), (0,255,0), 2)  # y-axis (green)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[3].ravel()), (255,0,0), 2)  # z-axis (blue)

    return img

def aruco_display(corners, ids, rejected, image):  
	if len(corners) > 0:
		
		ids = ids.flatten()
		
		for (markerCorner, markerID) in zip(corners, ids):
			
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners
			
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
			
			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			print("[Inference] ArUco marker ID: {}".format(markerID))
			
	return image

def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)  # Define aruco_dict here
    parameters = cv2.aruco.DetectorParameters()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Call aruco_display to draw markers and their IDs
    frame = aruco_display(corners, ids, rejected_img_points, frame)

    if len(corners) > 1:
        print("There are two or more markers")
        # Estimate pose for the first marker
        rvec1, tvec1, _ = cv2.aruco.estimatePoseSingleMarkers(corners[0], 0.175, matrix_coefficients, distortion_coefficients)

        # Estimate pose for the second marker
        rvec2, tvec2, _ = cv2.aruco.estimatePoseSingleMarkers(corners[1], 0.175, matrix_coefficients, distortion_coefficients)
        
        # Calculating Relative Positions
        distance_vector = tvec1-tvec2;
        distance = distance_between_markers(tvec1.squeeze(), tvec2.squeeze())
        print("Distance between markers:", distance)
        print("Distance vector between markers:", distance_vector)
        
        # Calculating Relative Rotations
        Theta = angle_between_markers(rvec1.squeeze(), rvec2.squeeze())  
        #print(f"Relative Rotation Angle around X-axis (degrees): {Theta[0]}")
        #print(f"Relative Rotation Angle around Y-axis (degrees): {Theta[1]}")
        print(f"Relative Rotation Angle around Z-axis (degrees): {Theta[2]}")
        
        cv2.aruco.drawDetectedMarkers(frame, corners)
        draw_axis(frame, rvec1, tvec1, matrix_coefficients, distortion_coefficients, 0.1)
        draw_axis(frame, rvec2, tvec2, matrix_coefficients, distortion_coefficients, 0.1)

    return frame

def distance_between_markers(tvec1, tvec2):
    # Calculate the Euclidean distance between two translation vectors
    distance = np.linalg.norm(tvec1 - tvec2)
    return distance

def angle_between_markers(rvec1, rvec2):
    # Calculate relative rotation vectors
    relative_rvec1 = rvec1 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)
    relative_rvec2 = rvec2 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)

    # Convert relative rotation vectors to rotation matrices
    rotation_matrix1, _ = cv2.Rodrigues(relative_rvec1)
    rotation_matrix2, _ = cv2.Rodrigues(relative_rvec2)

    # Compute the rotation difference between the two markers
    relative_rotation_matrix = np.dot(rotation_matrix2, np.linalg.inv(rotation_matrix1))
        
    # Convert rotation matrix to Euler angles
    relative_euler_angles, _ = cv2.Rodrigues(relative_rotation_matrix)
     
    # Convert to Angles in x, y, and z
    theta_x = np.arctan2(relative_rotation_matrix[2, 1], relative_rotation_matrix[2, 2])
    theta_y = np.arctan2(-relative_rotation_matrix[2, 0], np.sqrt(relative_rotation_matrix[2, 1]**2 + relative_rotation_matrix[2, 2]**2))
    theta_z = np.arctan2(relative_rotation_matrix[1, 0], relative_rotation_matrix[0, 0])
    
    # Convert to degrees
    theta_x_deg = np.degrees(theta_x)
    theta_y_deg = np.degrees(theta_y)
    theta_z_deg = np.degrees(theta_z)
    
    # Make it a single matrix
    Theta = np.array([theta_x_deg, theta_y_deg, theta_z_deg])
        
    return Theta

aruco_type = "DICT_ARUCO_ORIGINAL"

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))


cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Iinitializing Coordinate Vectors
tvec1 = np.array([0, 0, 0])
tvec2 = np.array([0, 0, 0])
rvec1 = np.array([0, 0, 0])
rvec2 = np.array([0, 0, 0])

# While loop for computer vision, currently calculates distance between markers only
while cap.isOpened():
     
    ret, img = cap.read()
    
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)
    
    cv2.imshow('Estimated Pose', output)
   
    # Get the corners and ids of detected markers
    corners, ids, _ = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7789170096938656
Distance vector between markers: [[[-0.75325535  0.06626705  0.18688704]]]
Relative Rotation Angle around Z-axis (degrees): -8.915823894207591
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7859798085697877
Distance vector between markers: [[[-0.7612644   0.06532587  0.18431848]]]
Relative Rotation Angle around Z-axis (degrees): 13.67715893188025
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7859798085697877
Distance vector between markers: [[[-0.7612644   0.06532587  0.18431848]]]
Relative Rotation Angle around Z-axis (degrees): 13.67715893188025
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7859798085697877
Distance vector b

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7859798085697877
Distance vector between markers: [[[-0.7612644   0.06532587  0.18431848]]]
Relative Rotation Angle around Z-axis (degrees): 13.67715893188025
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7859798085697877
Distance vector between markers: [[[-0.7612644   0.06532587  0.18431848]]]
Relative Rotation Angle around Z-axis (degrees): 13.67715893188025
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7859798085697877
Distance vector between markers: [[[-0.7612644   0.06532587  0.18431848]]]
Relative Rotation Angle around Z-axis (degrees): 13.67715893188025
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7859798085697877
Distance vector be

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7994845112513979
Distance vector between markers: [[[-0.77072281  0.06696342  0.20168722]]]
Relative Rotation Angle around Z-axis (degrees): 10.8062074901871
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.798169897468736
Distance vector between markers: [[[-0.77985589  0.05540639  0.16071748]]]
Relative Rotation Angle around Z-axis (degrees): -10.000305814640832
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.7971438908481423
Distance vector between markers: [[[-0.76883357  0.063243    0.2008324 ]]]
Relative Rotation Angle around Z-axis (degrees): -8.33229916094751
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.82

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8249320156846575
Distance vector between markers: [[[-0.81084641  0.04645217  0.14450995]]]
Relative Rotation Angle around Z-axis (degrees): 1.9524665538678445
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.82441700930991
Distance vector between markers: [[[-0.81130729  0.04656476  0.13883662]]]
Relative Rotation Angle around Z-axis (degrees): 1.6186913699548264
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8251961554180749
Distance vector b

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.821087776991667
Distance vector between markers: [[[-0.8127088   0.04339033  0.10865924]]]
Relative Rotation Angle around Z-axis (degrees): -2.455011017513261
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8235068146296851
Distance vector between markers: [[[-0.81290096  0.0428885   0.12456354]]]
Relative Rotation Angle around Z-axis (degrees): -4.556925874607406
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector 

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8225510300813321
Distance vector between markers: [[[-0.81277096  0.04301566  0.11892526]]]
Relative Rotation Angle around Z-axis (degrees): -4.981603409252881
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8225510300813321
Distance vector between markers: [[[-0.81277096  0.04301566  0.11892526]]]
Relative Rotation Angle around Z-axis (degrees): -4.981603409252881
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8225510300813321
Distance vector

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8235068146296851
Distance vector

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8231661883998129
Distance vector between markers: [[[-0.80978414  0.04669303  0.14025686]]]
Relative Rotation Angle around Z-axis (degrees): 0.2767827595595182
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.821087776991667
Distance vector between markers: [[[-0.8127088   0.04339033  0.10865924]]]
Relative Rotation Angle around Z-axis (degrees): -2.455011017513261
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8226512353211006
Distance vector between markers: [[[-0.81477215  0.04409523  0.10467575]]]
Relative Rotation Angle around Z-axis (degrees): -4.821513456988243
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8235068146296851
Distance vector 